In [1]:
%%capture
!pip install diffusers
!pip install transformers scipy ftfy accelerate
!pip install stable_diffusion_videos

In [2]:
connect_google_drive = True #@param {type:"boolean"}

if connect_google_drive:
    from google.colab import drive

    drive.mount('/content/gdrive')

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)


Mounted at /content/gdrive


In [3]:
import os 
import pandas as pd
import numpy as np

code_folder = r'/content/gdrive/MyDrive/AI Music Visuals Share/New Codes'

prompt_sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1YhC-iejPPC0bV4OJlxqZIQJj3FJqM-6xYauY7TB8EQA')

sheets = prompt_sheet.worksheet('prompts').get_all_values()
df_prompt = pd.DataFrame(sheets[1:], columns=sheets[0]).set_index('name')


In [4]:

import torch
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4",
                                               torch_dtype=torch.float16,
                                               safety_checker=None
                                               )  


pipe = pipe.to("cuda")

from PIL import Image

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid


Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [7]:
df_prompt

,prompt,seeds
name,,
succulent,"a giant succulent in the desert in the sand, t...",5842234013956479
reflected galaxy,underground lake with a galaxy reflected in th...,5883559517894420
red cave1,"descending deep into a cave, red lava on the w...",4866412378884770 4940561922459094


In [8]:
prompt_name = 'succulent'
prompt = df_prompt['prompt'][prompt_name]
seed = 5842234013956479

width = 512
height = 512

generator = torch.Generator(device="cuda")

generator.manual_seed(seed)

latent_choose = torch.randn(
    (1, pipe.unet.in_channels, height // 8, width // 8),
    generator = generator,
    device = "cuda"
)


In [13]:
code_folder = r'/content/gdrive/MyDrive/AI Music Visuals Share/New Codes'
output_dir = os.path.join(code_folder, 'output_latent_shift', "{}_{}".format(prompt_name, seed))
if not os.path.exists(output_dir): os.makedirs(output_dir)

latent_shifts = np.linspace(0, 0.1, 8)

for i,latent_shift in enumerate(latent_shifts):

  latents_shifted = latent_choose + latent_shift

  with torch.autocast("cuda"):
      images = pipe(
          [prompt] * len(latents_shifted),
          guidance_scale=5,
          latents = latents_shifted,
      )

  output_image = images.images[0]

  # for i, image in enumerate(images.images):
  output_image.save(os.path.join(output_dir, '{0:04d}.png'.format(i)))

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [14]:
from stable_diffusion_videos import make_video_pyav

make_video_pyav(output_dir, 
                output_filepath=os.path.join(code_folder, 'output_latent_shift', "{}.mp4".format(seed)),
                fps=10
                )

'/content/gdrive/MyDrive/AI Music Visuals Share/New Codes/output_latent_shift/5842234013956479.mp4'